In [27]:
import numpy as np , os
import tensorflow as tf
import collections
import pickle
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input,Dropout
import os
import re

gpus = tf.config.list_physical_devices('GPU')
tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [28]:
questions = []
answers = []

folder_path = 'data_generation/newQuestions'  # Geben Sie den Pfad zum gewünschten Ordner an

# Durchlaufen aller Dateien im Ordner
que = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path)]
folder_path = 'data_generation/newAnswers' 
ans = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path)]
print(len(que))
print(len(ans))
#my better version
file_path = './processed_data/'

with open(file_path+'all_fragen.txt', 'r') as inputfile:
    questions += [line.rstrip() for line in inputfile.readlines()]
    

with open(file_path+'all_antworten.txt', 'r') as inputfile:
    answers += [line.rstrip() for line in inputfile.readlines()]

def make_clean(sentences):
    result = []
    for sentence in sentences:
        txt = sentence.lower()
        txt = re.sub(r"i'm", "i am", txt)
        txt = re.sub(r"he's", "he is", txt)
        txt = re.sub(r"she's", "she is", txt)
        txt = re.sub(r"that's", "that is", txt)
        txt = re.sub(r"what's", "what is", txt)
        txt = re.sub(r"where's", "where is", txt)
        txt = re.sub(r"\'ll", " will", txt)
        txt = re.sub(r"\'ve", " have", txt)
        txt = re.sub(r"\'re", " are", txt)
        txt = re.sub(r"\'d", " would", txt)
        txt = re.sub(r"won't", "will not", txt)
        txt = re.sub(r"can't", "can not", txt)
        txt = re.sub(r"[^\w\s]", "", txt)
        result += [txt]
    return result

#print(antworten)
clean_ques = make_clean(questions)
clean_ans = make_clean(answers)
###  count occurences ###
counter_words = {}

for line in clean_ques:
    for word in line.split():
        if word not in counter_words:
            counter_words[word] = 1
        else:
            counter_words[word] += 1
for line in clean_ans:
    for word in line.split():
        if word not in counter_words:
            counter_words[word] = 1
        else:
            counter_words[word] += 1
vocab = {}
word_num = 0
for word, count in counter_words.items():
    vocab[word] = word_num
    word_num += 1
print(vocab)
print(len(vocab))    

for i in range(len(clean_ans)):
    clean_ans[i] = '<SOS> ' + clean_ans[i] + ' <EOS>'

tokens = ['<PAD>', '<EOS>', '<OUT>', '<SOS>']
x = len(vocab)
print(x)
for token in tokens:
    vocab[token] = x
    x += 1
    
    
vocab['cameron'] = vocab['<PAD>']
vocab['<PAD>'] = 0
print(len(vocab))

### inv answers dict ###
inv_vocab = {w:v for v, w in vocab.items()}

encoder_inp = []
for line in clean_ques:
    lst = []
    for word in line.split():
        if word not in vocab:
            lst.append(vocab['<OUT>'])
        else:
            lst.append(vocab[word])
        
    encoder_inp.append(lst)

decoder_inp = []
for line in clean_ans:
    lst = []
    for word in line.split():
        if word not in vocab:
            lst.append(vocab['<OUT>'])
        else:
            lst.append(vocab[word])        
    decoder_inp.append(lst)

### delete
del(clean_ans, clean_ques, line, lst, word)    

longest_sequence = 0
for x in encoder_inp+decoder_inp:
    
    if len(x) > longest_sequence:
        longest_sequence = len(x)     
print(longest_sequence) 
from tensorflow.keras.preprocessing.sequence import pad_sequences
encoder_inp = pad_sequences(encoder_inp, longest_sequence, padding='post', truncating='post')
decoder_inp = pad_sequences(decoder_inp, longest_sequence, padding='post', truncating='post')

def make_vector_to_sequence(input, dic):
    ints = []
    for k in input:
        try:
            ints.append(inv_vocab[k])
        except Exception as e:
            ints.append(inv_vocab[2])
        
    return ints    
decoder_final_output = []
for i in decoder_inp:
    decoder_final_output.append(i[1:]) 

decoder_final_output = pad_sequences(decoder_final_output, longest_sequence, padding='post', truncating='post')
from tensorflow.keras.utils import to_categorical

num_categories = len(vocab)  # Check the actual size of your vocabulary

# Debugging statements
#print("Number of categories:", num_categories)
#print("Decoder final output shape:", decoder_final_output.shape)
#print("Maximum value in decoder final output:", decoder_final_output.max())

# Convert to one-hot encoded representation
decoder_final_output = to_categorical(decoder_final_output, num_categories)

#print(decoder_final_output.shape)
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input

# for both

enc_inp = Input(shape=(longest_sequence, ))
dec_inp = Input(shape=(longest_sequence, ))
# embedding layer reduces dimensionality
# Compress output dim -> voc 722  bzw 300 
VOCAB_SIZE = len(vocab)

33
33
{'what': 0, 'is': 1, 'tacotron': 2, '2': 3, 'the': 4, 'architecture': 5, 'of': 6, 'how': 7, 'does': 8, 'use': 9, 'deep': 10, 'learning': 11, 'generate': 12, 'speech': 13, 'difference': 14, 'between': 15, 'and': 16, 'are': 17, 'some': 18, 'limitations': 19, 'handle': 20, 'different': 21, 'languages': 22, 'accents': 23, 'background': 24, 'noise': 25, 'speaking': 26, 'styles': 27, 'emotions': 28, 'in': 29, 'role': 30, 'textencoder': 31, 'spectral': 32, 'encoder': 33, 'wavenet': 34, 'vocoder': 35, 'pitch': 36, 'prosody': 37, 'attention': 38, 'mechanism': 39, 'long': 40, 'sentences': 41, 'rare': 42, 'words': 43, 'or': 44, 'proper': 45, 'nouns': 46, 'melspectrogram': 47, 'diacritics': 48, 'encoderdecoder': 49, 'postnet': 50, 'synthesis': 51, 'for': 52, 'genders': 53, 'ages': 54, 'f0': 55, 'conditioning': 56, 'duration': 57, 'model': 58, 'dialects': 59, 'language': 60, 'meldecoder': 61, 'rates': 62, 'characterembedding': 63, 'convolutional': 64, 'layers': 65, 'voices': 66, 'lstm': 67, '

In [29]:
def create_enc_model(encoded_states,modelname):
        # Load the encoder states
    with open("new_saves/"+ 'encoder_states.pkl', 'rb') as f:
        loaded_enc_states = pickle.load(f)

    # Load the model (including architecture and weights)
    loaded_model = tf.keras.models.load_model("new_saves/"+ 'model.h5')

    # Define the decoder input and states input layers
    decoder_state_input_h = Input(shape=(800,))
    decoder_state_input_c = Input(shape=(800,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

    # Get the relevant layers from the loaded model
    dec_embed = loaded_model.get_layer('embedding_2')  # Use the correct name for the embedding layer
    dec_lstm = loaded_model.get_layer('lstm_6') 

    # Create the decoder model
    decoder_outputs, state_h, state_c = dec_lstm(dec_embed.output, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]

    loaded_dec_model = Model([dec_lstm.input] + decoder_states_inputs,
                            [decoder_outputs] + decoder_states)
    return loaded_dec_model

In [ ]:
enc_model = Model([enc_inp], enc_states)

# decoder Model
decoder_state_input_h = Input(shape=(800,))
decoder_state_input_c = Input(shape=(800,))

decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# wir verwenden LSTM units mit den trainierten states
# h war hidden state mit Information Kontext 
# c cell state mit Inforamtion über Abhängigkeiten
decoder_outputs, state_h, state_c = dec_lstm(dec_embed , 
                                    initial_state=decoder_states_inputs)

decoder_states = [state_h, state_c]

dec_model = Model([dec_inp]+ decoder_states_inputs,
                                    [decoder_outputs]+ decoder_states)

In [ ]:
enc_inp = Input(shape=(longest_sequence, ))
dec_inp = Input(shape=(longest_sequence, ))

In [36]:
def get_enc_and_dec_model(encoded_states,modelname):
        # Load the encoder states
    with open("new_saves/"+ 'encoder_states.pkl', 'rb') as f:
        loaded_enc_states = pickle.load(f)

    # Load the model (including architecture and weights)
    loaded_model = tf.keras.models.load_model("new_saves/"+ 'model.h5')
    
    enc_model = Model([enc_inp], loaded_enc_states)

    # Get the relevant layers from the loaded model
    enc_lstm = loaded_model.get_layer('lstm_6')        # Use the correct name for the encoder LSTM layer
    dec_lstm = loaded_model.get_layer('lstm_7')        # Use the correct name for the decoder LSTM layer
    embed = loaded_model.get_layer('embedding_2')      # Use the correct name for the embedding layer

    # Create the encoder model
    enc_states = [Input(shape=(800,)), Input(shape=(800,))]
    enc_op, h, c = enc_lstm(embed.output, initial_state=enc_states)
    enc_states = [h, c]
    

    # Create the decoder model
    decoder_state_input_h = Input(shape=(800,))
    decoder_state_input_c = Input(shape=(800,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

    decoder_outputs, state_h, state_c = dec_lstm([embed.output] + decoder_states_inputs,
                                                initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    dec_model = Model([embed.input] + decoder_states_inputs,
                    [decoder_outputs] + decoder_states)
    return enc_model,dec_model

In [37]:
enc_model,dec_model = get_enc_and_dec_model("","")

ValueError: Found input tensor cannot be reached given provided output tensors. Please make sure the tensor KerasTensor(type_spec=TensorSpec(shape=(None, 800), dtype=tf.float32, name='input_20'), name='input_20', description="created by layer 'input_20'") is included in the model inputs when building functional model.

In [31]:
def create_pred_input(sentence):
    X_in = []
    for word in sentence.lower().split():
        try:
            X_in.append(vocab[word])
        except:
            X_in.append(vocab['<OUT>'])
            pass
    print(X_in)    
    return pad_sequences([X_in], longest_sequence, padding='post')

input_for_prediction = create_pred_input('What is Tacotron 2 ?')

[0, 1, 2, 3, 719]


In [32]:
def predict_sentence(sentences):
    sentences = make_clean(sentences)
    for sentence in sentences:
        print("Question  : " + sentence + "/n")
        pred_input = create_pred_input(sentence)
        stat = enc_model.predict(pred_input)
        print("make predicted Input for " + str(pred_input))
        empty_target_seq = np.zeros( ( 1 , 1) )
        ##   empty_target_seq = [0]
        empty_target_seq[0, 0] = vocab['<SOS>']       

        stop_condition = False
        decoded_translation = ''
        while not stop_condition :

            dec_outputs , h, c= dec_model.predict([ empty_target_seq] + stat )
            decoder_concat_input = dense(dec_outputs)
            ## decoder_concat_input = [0.1, 0.2, .4, .0, ...............]

            sampled_word_index = np.argmax( decoder_concat_input[0, -1, :] )
            ## sampled_word_index = [2]

            sampled_word = inv_vocab[sampled_word_index] + ' '

            ## inv_vocab[2] = 'hi'
            ## sampled_word = 'hi '

            if sampled_word != '<EOS> ':
                decoded_translation += sampled_word  

            if sampled_word == '<EOS> ' or len(decoded_translation.split()) > longest_sequence:
                stop_condition = True 

            empty_target_seq = np.zeros( ( 1 , 1 ) )  
            empty_target_seq[ 0 , 0 ] = sampled_word_index
            ## <SOS> - > hi
            ## hi --> <EOS>
            stat = [h, c]  

        print("chatbot : ", decoded_translation )
        print("============================================== \n")  

In [33]:
Testset = ["How does Tacotron 2 utilize a Sequence-to-Sequence architecture?",
"What is the purpose of the Spectral Encoder in Tacotron 2?",
"How does Tacotron 2 generate speech from spectrograms?"
"What is the role of the WaveNet Vocoder in Tacotron 2",
"How are character embeddings used in Tacotron 2?",
"What are the advantages of using convolutional networks in Tacotron 2?"]
#print(])
predict_sentence([questions[0],questions[1],questions[2]])
#predict_sentence(answers[0])
Testset = ["Can the Tacotron 2 deal with speech",
           "Can it generate speech",
           "what are current challenges",
           "Is it a sequence2sequ"
           ]
predict_sentence(Testset)
Testset = ["Is Tacotron an apple ?"]
predict_sentence(Testset)

Question  : what is tacotron 2/n
[0, 1, 2, 3]


ValueError: in user code:

    File "c:\Users\Ben\anaconda3\envs\Test2\lib\site-packages\keras\engine\training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\Ben\anaconda3\envs\Test2\lib\site-packages\keras\engine\training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Ben\anaconda3\envs\Test2\lib\site-packages\keras\engine\training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\Ben\anaconda3\envs\Test2\lib\site-packages\keras\engine\training.py", line 2111, in predict_step
        return self(x, training=False)
    File "c:\Users\Ben\anaconda3\envs\Test2\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\Ben\anaconda3\envs\Test2\lib\site-packages\keras\engine\input_spec.py", line 219, in assert_input_compat